# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
import math

# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = lines[1].split(" ")[-5] #la quantità è sempre la quintultima parola
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        #sono le prime due parole dell'ultima riga (sesta riga, quindi 5)
        self.customer = lines[5].split(" ")[-3:-1]#per ora mi fa una lista con nome e cognome 
        self.customer=self.customer[0]+" "+self.customer[1]#trasformo in una stringa unica
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = lines[3].split()[-1]
        self.bid_price = float(self.bid_price.split("€")[-2])#tolgo il simbolo €
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        ######## SCRIVI CODICE (Punto 2.) ########
        if self.bid_price>=ask_price:
            return ask_price
        else:
            return 0;
        #questa non serve a niente, non la uso mai, boh
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, contakm):
        ######## SCRIVI CODICE (Punto 2.) ########
        if contakm: #versione che tiene conto dei km
            if self.bid_price >= ask_price:
                if int(self.quantity) >= 100:
                    cost_per_km = cost_at_least_100
                else:
                    cost_per_km = cost_less_100    
                return (ask_price - production_cost)*int(self.quantity) - cost_per_km*self.Distance()
            return 0
        else:#versione che NON tiene conto dei km
            if self.bid_price >= ask_price:
                return (ask_price - production_cost)*int(self.quantity)
            return 0
            
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    def Distance2(self, starting_GCS = GCS_Piazza_Leo):
        # tentar non nuoce...
        
        # degrees -> radians
        # final destination point = [phi1, lambda1]
        # base point = [phi2, lambda2]
        phi1 = math.radians(self.GCS[0])
        lambda1 = math.radians(self.GCS[1])
        phi2 = math.radians(starting_GCS[0])
        lambda2 = math.radians(starting_GCS[1])
        
        # Ellipsoidal surface
        
        # Bowring's method for short lines:
        
        # needed data:
        # a (equatorial radius): 6 378 137.0 m
        a = 6378.137
        # b (polar radius): 6 356 752.3142 m
        b = 6356.7523142
        #1/f (inverse flattening): 298.257 223 563
        f = 1/298.257223563
       
        #second eccentricity squared e'^2
        e2 = f*(2-f)/(1-f)**2
        
        A=(1+e2*(math.cos(phi1)**4))**(1/2)
        B=(1+e2*(math.cos(phi1)**2))**(1/2)
        
        # shperical radius
        R=(((1+e2)**(1/2))/B**2)*a
        
        # phi1' phi2'
        phi1_1 = math.atan(math.tan(phi1)/B)
        phi2_1 = math.atan(math.tan(phi2)/B)
        
        # Deltas:
        D_phi = abs(phi2-phi1)
        D_lambda = abs(lambda2-lambda1)
        
        D_phi_1 = D_phi/B*(1+(3*e2)/(4*B**2)*D_phi*math.sin(2*phi1+(2/3)*D_phi))
        D_lambda_1 = A*D_lambda
        
        # Law of cosines -> inverse law of haversine (when distance is small):
        d = 2*R * math.asin( ( (math.sin((D_phi_1)/2))**2 + math.cos(phi1_1)*math.cos(phi2_1) * (math.sin((D_lambda_1)/2))**2 )**(1/2) )
        """
        https://en.wikipedia.org/wiki/Geographical_distance
        https://en.wikipedia.org/wiki/Great-circle_navigation
        
        https://en.wikipedia.org/wiki/Spherical_law_of_cosines
        https://en.wikipedia.org/wiki/Haversine_formula
        """
        return d

    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    ######## SCRIVI CODICE ########
    # Apri il file ... orders.append( ... Chiudi il file
    file = open(str(i+1)+'.txt','r') 
    orders.append(Order(file.read()))
    file.close()

for i in range(num_orders): orders[i].Details() #fin qui tutto ok

Maurizio Genalti ha ordinato 383 magliette taglia L a 13.35€ all'indirizzo [44.730801, 10.411061]
Enrico Zacchei ha ordinato 33 magliette taglia S a 19.16€ all'indirizzo [45.386683, 8.176936]
Bressan e ha ordinato 316 magliette taglia L a 19.58€ all'indirizzo [46.389283, 11.742332]
Fedeli e ha ordinato 255 magliette taglia M a 16.6€ all'indirizzo [44.983867, 9.346757]
Cerisola Azienda ha ordinato 389 magliette taglia M a 11.37€ all'indirizzo [45.219274, 10.508013]
Valentina Calcaterra ha ordinato 398 magliette taglia S a 6.98€ all'indirizzo [46.078556, 10.47238]
JP Iacovelli ha ordinato 29 magliette taglia XXL a 19.96€ all'indirizzo [46.035413, 6.990662]
Calcaterra Strumenti ha ordinato 307 magliette taglia M a 15.22€ all'indirizzo [45.900417, 11.562795]
JP Bacchiocchi ha ordinato 384 magliette taglia M a 7.42€ all'indirizzo [45.952838, 8.110702]
Gianmarco Sosta ha ordinato 357 magliette taglia XS a 15.86€ all'indirizzo [45.804997, 8.586271]
Gianmarco Carrara ha ordinato 293 magliette 

Manuel Calcaterra ha ordinato 43 magliette taglia XXL a 6.37€ all'indirizzo [45.383853, 11.645055]
Francesco Baroni ha ordinato 52 magliette taglia XS a 10.57€ all'indirizzo [45.101491, 9.361921]
MC Carrara ha ordinato 235 magliette taglia XL a 9.9€ all'indirizzo [46.316984, 7.2826]
Sartor Strumenti ha ordinato 289 magliette taglia L a 14.19€ all'indirizzo [45.895956, 6.644199]
Alessandro Sartor ha ordinato 53 magliette taglia XXL a 7.9€ all'indirizzo [45.193414, 10.601173]
Manuel Bressan ha ordinato 351 magliette taglia M a 8.46€ all'indirizzo [46.393007, 10.853974]
Davide Sosta ha ordinato 333 magliette taglia XL a 11.81€ all'indirizzo [44.534782, 9.489222]
Mirko Bacchiocchi ha ordinato 307 magliette taglia XS a 15.83€ all'indirizzo [46.323737, 10.584971]
Iacovelli e ha ordinato 108 magliette taglia XXL a 12.23€ all'indirizzo [45.325776, 8.33967]
Gianmarco Sgarbossa ha ordinato 494 magliette taglia XL a 5.54€ all'indirizzo [44.773254, 7.165038]
Enrico Sgarbossa ha ordinato 14 magliet

Bacchiocchi e ha ordinato 114 magliette taglia S a 6.14€ all'indirizzo [45.330933, 9.733152]
Bacchiocchi Giochi ha ordinato 125 magliette taglia S a 10.99€ all'indirizzo [45.790945, 9.437917]
Augusto Sosta ha ordinato 74 magliette taglia XXL a 15.89€ all'indirizzo [45.481015, 9.410362]
Giulia Carrara ha ordinato 487 magliette taglia L a 12.46€ all'indirizzo [46.566276, 6.786079]
Moneta Azienda ha ordinato 490 magliette taglia L a 11.46€ all'indirizzo [46.521621, 10.474226]
Vitellio e ha ordinato 280 magliette taglia XS a 16.99€ all'indirizzo [46.215419, 11.450555]
Sgarbossa Strumenti ha ordinato 396 magliette taglia M a 12.56€ all'indirizzo [44.479378, 11.26807]
JP Sgarbossa ha ordinato 237 magliette taglia S a 7.74€ all'indirizzo [45.963189, 12.155281]
JP Vitellio ha ordinato 231 magliette taglia M a 11.82€ all'indirizzo [45.0728, 9.995119]
Francesco Bressan ha ordinato 431 magliette taglia S a 13.45€ all'indirizzo [45.09102, 8.610417]
Filippo Zacchei ha ordinato 353 magliette taglia 

In [2]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
######## SCRIVI CODICE ######## 
possible_profits = [sum(j.Profit(i.bid_price,0) for j in orders) for i in orders]
_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
######## SCRIVI CODICE ########
max_profit = max(possible_profits)
print("Il massimo profitto è "+str(max_profit)+" €")

# Determina il prezzo migliore
######## SCRIVI CODICE ########
#il migliore degli ask_prices è quello a cui corrisponde il max_profit
for i in range(len(possible_profits)):
    if possible_profits[i]==max_profit:
        best_price=orders[i].bid_price
        break

print("Il miglior prezzo è "+str(best_price)+" €")

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price,0) > 0]
print("Si eseguiranno", len(orders_to_go),"ordini")

354564.00000000023
111280.95999999999
75871.46
264236.80000000005
330528.87000000005
90644.1200000001
3783.1600000000003
333773.2199999999
127616.81999999996
300749.72000000015
319720.09999999974
297624.0600000001
292184.63999999984
320043.81000000006
145445.52000000002
278994.62999999995
72250.16000000005
350567.8200000002
41209.04
42498.62000000001
297624.0600000001
221133.12
250398.00000000003
359609.5799999998
301417.4800000001
74561.78000000007
336214.0
166927.8599999998
141438.5
321325.50000000023
220224.95999999996
236757.35999999993
79222.64000000001
50923.07999999999
58505.99999999994
131947.55
354482.33999999985
354646.7199999999
320449.26000000024
222241.36
32068.160000000025
53982.5
349029.89999999973
346361.3999999999
-96508.80000000012
347993.25
172987.19999999998
331498.0400000001
167804.0
291512.5999999998
133508.98
142420.6
175307.03999999986
229734.19999999984
269949.3500000001
148226.5199999999
254089.45
281013.47999999975
-80335.04000000008
303529.5600000001
353913.

In [3]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
######## SCRIVI CODICE ########
possible_profits = [sum(j.Profit(i.bid_price,1) for j in orders) for i in orders] #cambia solo il contakm

_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)
print("Il massimo profitto è "+str(max_profit)+" €")

# Determina il prezzo migliore
######## SCRIVI CODICE ########
for i in range(len(possible_profits)):
    if possible_profits[i]==max_profit:
        best_price=orders[i].bid_price
        break

print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
######## SCRIVI CODICE ########          
orders_to_go = [ord1 for ord1 in orders if ord1.Profit(best_price,1) > 0] #cambia solo il contakm

print("Si eseguiranno", len(orders_to_go),"ordini")

######## SCRIVI CODICE ########
# ... assembla la lista degli ordini che devono che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto
######## SCRIVI CODICE ########



# creo vettore dei profitti degli ordini e lavoro su quello per ottimizzare le chiamate a Profit()
orders_profit = [orders_to_go[i].Profit(best_price,1) for i in range(len(orders_to_go))]

# bubble sorting
k=True
while k:
    k=False
    for i in range(len(orders_profit)-1):
        if orders_profit[i] < orders_profit[i+1]:
            #scambio
            orders_profit[i], orders_profit[i+1] = orders_profit[i+1], orders_profit[i]
            orders_to_go[i], orders_to_go[i+1] = orders_to_go[i+1], orders_to_go[i]
            #print("scambio " + str(i) + "<->" + str(i+1))
            k=True #è stato eseguito almeno uno scambio

_ = [otg.Details() for otg in orders_to_go]

Il massimo profitto è 257595.32835514666 €
Il miglior prezzo è €15.03
Si eseguiranno 130 ordini
Francesco Sosta ha ordinato 494 magliette taglia L a 15.6€ all'indirizzo [44.958786, 10.107809]
Gianmarco Bressan ha ordinato 491 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
Carrara Servizi ha ordinato 486 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]
Manuel Carrara ha ordinato 492 magliette taglia XS a 19.85€ all'indirizzo [44.811402, 11.070849]
Gianmarco Fedeli ha ordinato 441 magliette taglia M a 16.77€ all'indirizzo [45.196751, 9.768684]
Davide Baroni ha ordinato 431 magliette taglia S a 19.62€ all'indirizzo [45.755771, 8.645121]
Iacovelli Azienda ha ordinato 497 magliette taglia S a 17.92€ all'indirizzo [45.058065, 6.422715]
Maurizio Baroni ha ordinato 467 magliette taglia XS a 15.28€ all'indirizzo [44.989011, 11.117416]
Filippo Iacovelli ha ordinato 467 magliette taglia L a 18.25€ all'indirizzo [46.042605, 11.075677]
Maurizio Fedeli ha ordinato 

In [4]:
#provo ad usare distance2, calcolo qualche distanza

for i in range(num_orders):
    print("Distanza ordine " + str(i+1) + " flat surface: \t" + str(orders[i].Distance()) + " km")
    print("Distanza ordine " + str(i+1) + " ellipsoidal surface:\t " + str(orders[i].Distance2()) + " km")

# mi sembra che la mia funzione Distance2() fornisca dei numeri ragionevoli

Distanza ordine 1 flat surface: 	124.13702899189978 km
Distanza ordine 1 ellipsoidal surface:	 124.82106689794381 km
Distanza ordine 2 flat surface: 	82.53432774217602 km
Distanza ordine 2 ellipsoidal surface:	 82.79732634690099 km
Distanza ordine 3 flat surface: 	220.72852270755862 km
Distanza ordine 3 ellipsoidal surface:	 219.79395531228067 km
Distanza ordine 4 flat surface: 	55.649349916380785 km
Distanza ordine 4 ellipsoidal surface:	 55.72638160988014 km
Distanza ordine 5 flat surface: 	103.96444701741142 km
Distanza ordine 5 ellipsoidal surface:	 104.42466061743778 km
Distanza ordine 6 flat surface: 	117.80385793911813 km
Distanza ordine 6 ellipsoidal surface:	 117.62234690916802 km
Distanza ordine 7 flat surface: 	185.0817552818689 km
Distanza ordine 7 ellipsoidal surface:	 184.68731668213292 km
Distanza ordine 8 flat surface: 	188.12288716081324 km
Distanza ordine 8 ellipsoidal surface:	 187.8965519628786 km
Distanza ordine 9 flat surface: 	101.77798311073549 km
Distanza ordin

Distanza ordine 95 ellipsoidal surface:	 204.71189065481914 km
Distanza ordine 96 flat surface: 	101.8472408610678 km
Distanza ordine 96 ellipsoidal surface:	 102.05198203459643 km
Distanza ordine 97 flat surface: 	215.31719125977506 km
Distanza ordine 97 ellipsoidal surface:	 214.43282146136235 km
Distanza ordine 98 flat surface: 	117.11805640021917 km
Distanza ordine 98 ellipsoidal surface:	 117.34448674867973 km
Distanza ordine 99 flat surface: 	101.50410405494158 km
Distanza ordine 99 ellipsoidal surface:	 101.39772893311438 km
Distanza ordine 100 flat surface: 	195.68109570976316 km
Distanza ordine 100 ellipsoidal surface:	 196.85592282840807 km
Distanza ordine 101 flat surface: 	165.97281852541974 km
Distanza ordine 101 ellipsoidal surface:	 167.14205602905994 km
Distanza ordine 102 flat surface: 	143.94106771968148 km
Distanza ordine 102 ellipsoidal surface:	 143.5999961752283 km
Distanza ordine 103 flat surface: 	232.76992829913613 km
Distanza ordine 103 ellipsoidal surface:	 2

Distanza ordine 397 flat surface: 	105.71647416530344 km
Distanza ordine 397 ellipsoidal surface:	 106.02564933795918 km
Distanza ordine 398 flat surface: 	196.89856873567737 km
Distanza ordine 398 ellipsoidal surface:	 196.29411750200808 km
Distanza ordine 399 flat surface: 	67.64518407813105 km
Distanza ordine 399 ellipsoidal surface:	 67.75825254075758 km
Distanza ordine 400 flat surface: 	193.36636015055652 km
Distanza ordine 400 ellipsoidal surface:	 192.62994854278972 km
Distanza ordine 401 flat surface: 	127.37621612196793 km
Distanza ordine 401 ellipsoidal surface:	 127.91440762005807 km
Distanza ordine 402 flat surface: 	220.93094609963924 km
Distanza ordine 402 ellipsoidal surface:	 219.96433597669818 km
Distanza ordine 403 flat surface: 	156.9905873916927 km
Distanza ordine 403 ellipsoidal surface:	 156.64345262927282 km
Distanza ordine 404 flat surface: 	187.81359125213584 km
Distanza ordine 404 ellipsoidal surface:	 187.11517122037432 km
Distanza ordine 405 flat surface: 	